In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
# from langchain.storage import LocalFileStore

# cache_dir = LocalFileStore("./.cache/")
loader = UnstructuredFileLoader("../files/chapter_three.txt");
# loader.load()

In [ ]:
# RecursiveCharacterTextSplitter 

# splitter = RecursiveCharacterTextSplitter();
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50);

# len(loader.load_and_split(text_splitter=splitter))
loader.load_and_split(text_splitter=splitter)

In [ ]:
# CharacterTextSplitter

splitter2 = CharacterTextSplitter(separator="\n")
splitter2._chunk_size = 600
splitter2._chunk_overlap = 100

# len(loader.load_and_split(text_splitter=splitter2))
loader.load_and_split(text_splitter=splitter2)

In [ ]:
# tiktoken

splitter3 = CharacterTextSplitter.from_tiktoken_encoder(separator="\n")
splitter3._chunk_size = 600
splitter3._chunk_overlap = 100

# len(loader.load_and_split(text_splitter=splitter3))
loader.load_and_split(text_splitter=splitter3)

In [ ]:
# embedding

from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()
# embedder.embed_query("hi")
vector = embedder.embed_documents(["hi", "how", "are", "you and what is your name"])

# len(vector)
len(vector[0])
# vector

In [ ]:
# vector store

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

loader2 = UnstructuredFileLoader("../files/chapter_three.txt");

splitter4 = CharacterTextSplitter.from_tiktoken_encoder(separator="\n")
splitter4._chunk_size = 600
splitter4._chunk_overlap = 100

docs = loader2.load_and_split(text_splitter=splitter4)
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
# result = vectorstore.similarity_search("where does winston live?")
# len(result)
# result

from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("../.cache/")
cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
	embeddings, cache_dir
)

vectorstore = Chroma.from_documents(documents=docs, embedding=cache_embeddings)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# stuff
llm = ChatOpenAI()
chain = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=vectorstore.as_retriever(),
)

chain.run("where does winston live?")

'Winston lives in Oceania, which is a totalitarian state in George Orwell\'s novel "1984." He resides in a city under constant surveillance by the government, with telescreens in every building, including his apartment.'

In [9]:
# refine
chain_refine = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="refine",
	retriever=vectorstore.as_retriever(),
)

chain_refine.run("where does winstone live?")

'The additional context provided from the novel "1984" by George Orwell sheds light on Winston\'s current living situation. After being released from the Ministry of Love, Winston lives in a state of despair and degradation. He frequents a café where the staff cater to his every need and provide him with alcohol and a chessboard. Despite his physical decline and the constant surveillance of the Party, Winston finds solace in the belief that "White always mates," symbolizing the eternal triumph of Good over Evil.\n\nIn conclusion, Winston lives in a state of physical and emotional deterioration, surrounded by the oppressive presence of the Party and its surveillance. His daily routine at the café reflects his resignation to the Party\'s control and his attempts to find some semblance of normalcy in a world dominated by fear and oppression.'